In [1]:
from boknis_eck import BEDatabaseWizard
from spectrum import SfgAverager, DummyPlotter 

from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.lines import Line2D

%matplotlib qt
plt.style.use("qt.mpltstyle")

# plot functions
def plot_by_time(dataframes, param, leg, scale=1):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()

    months = MonthLocator(range(1, 13), bymonthday=1, interval=6)
    monthsFmt = DateFormatter("%b '%y")
   
    for df in dataframes:
        
        ax.scatter(df["sampling_date"], df["sml_"+param]*scale, color="red")
        ax.scatter(df["sampling_date"], df["bulk_"+param]*scale, color="blue", marker="^")
        ax2.scatter(df["sampling_date"], df["chlorophyll"], color="green", marker="+")

        ax.plot(df["sampling_date"], df["sml_"+param]*scale, color="red")
        ax.plot(df["sampling_date"], df["bulk_"+param]*scale, color="blue")
        ax2.plot(df["sampling_date"], df["chlorophyll"], color="green")


    for i in range(8, 20, 1):
            lower = date(2000+i, 3, 1)
            upper = date(2000 + i, 9, 1)
            ax.axvspan(lower, upper, color="gray", alpha=0.4)

    legend_elements = [Line2D([0], [0], marker='^', label='Bulk water',
                              markerfacecolor='blue', mew=0.3,  mec="blue", aa=True, linestyle=''),
                               Line2D([0], [0], marker='o', label='Surface microlayer',
                                       markerfacecolor='red', mew=0.3, mec="red", aa=True, linestyle=''),
                               Line2D([0], [0], marker='+', label='Chlorophyll a',
                                       markerfacecolor='green', mew=2, mec="green", aa=True, linestyle='', markersize=10)
                                ]

    ax.legend(handles=legend_elements)
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(monthsFmt)
    ax.autoscale_view()
    fig.autofmt_xdate()

    ax.set_xlabel("time ")
    ax.set_ylabel(leg)
    ax2.set_ylabel("Chlorophyll a concentration/\n µg/L")
    rcParams['xtick.labelsize'] = 'small'

def plot_ratio_by_time(dataframes, params):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()

    months = MonthLocator(range(1, 13), bymonthday=1, interval=6)
    monthsFmt = DateFormatter("%b '%y")
   
    for df in dataframes:
        
        if params[1] == "oh":
            sml_total_oh = df["sml_oh1"]+df["sml_oh2"]
            bulk_total_oh = df["bulk_oh1"]+df["bulk_oh2"]     
            sml_ratio = df["sml_"+params[0]]/sml_total_oh
            bulk_ratio = df["bulk_"+params[0]]/bulk_total_oh
            
        else: 
            sml_ratio = df["sml_"+params[0]]/df["sml_"+params[1]]
            bulk_ratio = df["bulk_"+params[0]]/df["bulk_"+params[1]]
        
        ax.scatter(df["sampling_date"], sml_ratio, color="red")
        ax.scatter(df["sampling_date"], bulk_ratio, color="blue", marker="^")
        ax2.scatter(df["sampling_date"], df["chlorophyll"], color="green", marker="+")

        #ax.plot(df["sampling_date"], sml_ratio, color="red")
        #ax.plot(df["sampling_date"], bulk_ratio, color="blue")
        #ax2.plot(df["sampling_date"], df["chlorophyll"], color="green")


    for i in range(8, 20, 1):
            lower = date(2000+i, 3, 1)
            upper = date(2000 + i, 9, 1)
            ax.axvspan(lower, upper, color="gray", alpha=0.4)

    legend_elements = [Line2D([0], [0], marker='^', label='Bulk water',
                              markerfacecolor='blue', mew=0.3,  mec="blue", aa=True, linestyle=''),
                               Line2D([0], [0], marker='o', label='Surface microlayer',
                                       markerfacecolor='red', mew=0.3, mec="red", aa=True, linestyle=''),
                               Line2D([0], [0], marker='+', label='Chlorophyll a',
                                       markerfacecolor='green', mew=2, mec="green", aa=True, linestyle='', markersize=10)
                                ]

    ax.legend(handles=legend_elements)
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(monthsFmt)
    ax.autoscale_view()
    fig.autofmt_xdate()

    ax.set_xlabel("time ")
    ax.set_ylabel(f'{params[0]}/{params[1]} ratio')
    ax2.set_ylabel("Chlorophyll a concentration/\n µg/L")
    rcParams['xtick.labelsize'] = 'small'

    plt.show()


bz = BEDatabaseWizard()


In [2]:
bz.filter_year(2018)

,id,sampling_date,bulk_no,sml_no,sml_coverage,sml_ch,sml_oh1,sml_oh2,sml_dangling,bulk_coverage,bulk_ch,bulk_oh1,bulk_oh2,bulk_dangling,chlorophyll
74,75,2018-03-05,3.0,6,0.280974,0.062303,0.0,0.0,0.000000,0.032193,0.013384,None,NaN,0.011060,NaN
75,76,2018-03-27,2.0,7,0.157280,0.021050,NaN,NaN,0.006456,0.150981,0.021908,None,NaN,0.002839,NaN
77,78,2018-08-01,1.0,5,0.000000,0.016367,NaN,NaN,0.009447,0.000000,0.018789,None,NaN,0.010213,NaN
78,79,2018-10-18,3.0,6,0.057841,0.028222,NaN,NaN,0.012689,0.052542,0.033364,None,NaN,0.014346,NaN
79,80,2018-12-13,3.0,6,0.084070,0.013385,NaN,NaN,0.015676,0.045382,0.014558,None,NaN,0.016473,NaN
80,81,2018-11-13,2.0,6,0.103374,0.012538,NaN,NaN,0.015786,0.083234,0.011701,None,NaN,0.016214,NaN
81,82,2018-06-21,1.0,1,0.053842,0.001160,NaN,NaN,0.015626,0.058601,0.001305,None,NaN,0.015320,NaN
82,83,2018-09-20,1.0,1,0.076673,0.004051,NaN,NaN,0.014008,0.000000,0.000000,None,NaN,0.013567,NaN


D:\Anaconda\lib\site-packages\pandas\plotting\_matplotlib\converter.py:102: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [4]:
# Testcode section
from orm import WorkDatabaseWizard
from spectrum import DummyPlotter

import matplotlib.pyplot as plt
%matplotlib qt
plt.style.use("jupno.mplstyle")

wz = WorkDatabaseWizard()

test = get_spectrum_by_property(wz, "surfactant", "BX12")

DummyPlotter(test[0:12]).plot_all()

In [5]:
a = get_be_spectra_monthwise(wz)

C:\Users\lange\Desktop\CharmingSFG\SFG\spectrum.py:91: RuntimeWarning: divide by zero encountered in true_divide
  self.normalized_intensity = self.raw_intensity / (self.vis_intensity * self.ir_intensity)
C:\Users\lange\Desktop\CharmingSFG\SFG\spectrum.py:91: RuntimeWarning: invalid value encountered in true_divide
  self.normalized_intensity = self.raw_intensity / (self.vis_intensity * self.ir_intensity)


In [6]:
avg_dic = {}
for key in a:
   avg = SfgAverager(a[key], enforce_scale=True)
   avg_dic[key] = avg.average_spectrum
for key in avg_dic:
        if key != 12:
            plt.plot(avg_dic[key].x, avg_dic[key].y, label=key, marker='^')

plt.legend()

plt.show()

D:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:1503: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')
D:\Anaconda\lib\site-packages\numpy\core\_methods.py:75: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [3]:
def enforce_base():
    reg1 = np.arange(2750, 3055, 5)
    reg2 = np.arange(3050, 3670, 20)
    reg3 = np.arange(3650, 3845, 5)
    new = np.concatenate((reg1, reg2, reg3), axis=None)
    return new

print(enforce_base())

xroot= enforce_base()
ax = [2750, 2755, 2780, 3050]
ay = [5, 15, 16.7, 3]

bx = [2750, 2755, 2780, 3050, 3070, 3500, 3600, 3650, 3700, 3820]
by= [27, 4, 15, 76, 34, 12, 5, 3, 3, 10]

ax_new = np.interp(xroot, ax, ay)
mask = (xroot > np.max(ax)) | (xroot < np.min(ax))
ax_new[mask] = np.nan

from scipy import integrate

q = integrate.simps(ax_new, xroot)
q


[2750 2755 2760 2765 2770 2775 2780 2785 2790 2795 2800 2805 2810 2815
 2820 2825 2830 2835 2840 2845 2850 2855 2860 2865 2870 2875 2880 2885
 2890 2895 2900 2905 2910 2915 2920 2925 2930 2935 2940 2945 2950 2955
 2960 2965 2970 2975 2980 2985 2990 2995 3000 3005 3010 3015 3020 3025
 3030 3035 3040 3045 3050 3050 3070 3090 3110 3130 3150 3170 3190 3210
 3230 3250 3270 3290 3310 3330 3350 3370 3390 3410 3430 3450 3470 3490
 3510 3530 3550 3570 3590 3610 3630 3650 3650 3655 3660 3665 3670 3675
 3680 3685 3690 3695 3700 3705 3710 3715 3720 3725 3730 3735 3740 3745
 3750 3755 3760 3765 3770 3775 3780 3785 3790 3795 3800 3805 3810 3815
 3820 3825 3830 3835 3840]


D:\Anaconda\lib\site-packages\scipy\integrate\quadrature.py:376: RuntimeWarning: divide by zero encountered in true_divide
  h0divh1 = h0 / h1
D:\Anaconda\lib\site-packages\scipy\integrate\quadrature.py:378: RuntimeWarning: divide by zero encountered in true_divide
  y[slice1]*hsum*hsum/hprod +
D:\Anaconda\lib\site-packages\scipy\integrate\quadrature.py:378: RuntimeWarning: invalid value encountered in add
  y[slice1]*hsum*hsum/hprod +


nan

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps as sp
from scipy.integrate import trapz as sp


x= [3005, 3010, 3015, 3020, 3025, 3030, 3035, 3040, 3045, 3050, 3050, 3070, 3090, 3110, 3130, 3150, 3170, 3190, 3210, 3230, 3250, 3270, 3290, 3310, 3330, 3350]
y= [0.0001556, 0.0001771, 0.00017038, 0.00016676, 0.00017883, 0.00015801, 0.0001725,  0.00018268, 0.00019829, 0.00017083, 0.00017083, 0.00018625, 0.00017146, 0.00019095, 0.00017107, 0.00021096, 0.00023474, 0.00035913, 0.00033207, 0.00036671, 0.00025562, 0.00021671, 0.00021296, 0.00017954, 0.00015827, 0.00016871]

print(sp(y, x))

0.076163025


NameError: name 'plt' is not defined